In [1]:
import pandas as pd 

# train_data.head(5)
def clean_data(df):
    #Replace the missing value in both fare and age column with thier median
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())

    #Replace the missing value in embarked col with "X"
    df['Embarked'] = df['Embarked'].fillna('X')

    df['Cabin'] = df['Cabin'].fillna('X') # Preenche ausentes com 'X'
    df['Cabin'] = df['Cabin'].str[0]     

    #Create another column "Title" 
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Mlle', 'Ms', 'Lady', 'Countess', 'Mme', 'Dona'], 'Miss')
    df['Title'] = df['Title'].replace(['Rev', 'Don', 'Capt', 'Major', 'Sir', 'Col', 'Jonkheer'], 'Rare')

    df['Name_Words'] = df['Name'].str.split().str.len()

    df['Is_Child'] = (df['Title'].isin(['Master', 'Miss'])) & (df['Age'] < 18)
    return df 
train_data = clean_data(pd.read_csv('train.csv'))
test_data =clean_data( pd.read_csv('test.csv'))
train_data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Name_Words,Is_Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,X,S,Mr,4,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,Mrs,7,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,X,S,Miss,3,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S,Mrs,7,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,X,S,Mr,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,X,S,Rare,3,False
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B,S,Miss,4,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,X,S,Miss,5,False
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C,Mr,4,False


In [2]:
from sklearn.model_selection import train_test_split
X= train_data.drop(columns=['PassengerId','Survived','Name'],axis=1)
y = train_data['Survived']

X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=.84, random_state=1)



In [3]:
from catboost import Pool, CatBoostClassifier, cv 
model = CatBoostClassifier(eval_metric='Accuracy', use_best_model=True,random_state=42,)
cat_features = ['Pclass','Sex','SibSp','Parch','Ticket','Cabin','Embarked','Title','Name_Words','Is_Child']
model.fit(X_train,y_train,cat_features=cat_features,eval_set=(X_val,y_val), silent=True)



In [4]:
feature_importance= model.get_feature_importance(Pool(X_train, label= y_train, cat_features=cat_features))
features_name = X_train.columns
feature_importance_df = pd.DataFrame({
    'Feature_importance':feature_importance,
    'Feature_name': features_name
})
feature_importance_df = feature_importance_df.sort_values(by="Feature_importance", ascending=False)
feature_importance_df

,Feature_importance,Feature_name
9,18.297695,Title
2,11.460228,Age
5,11.324273,Ticket
10,10.505984,Name_Words
0,10.192990,Pclass
7,9.560675,Cabin
6,7.848449,Fare
8,7.529689,Embarked
3,6.342526,SibSp
4,5.014161,Parch


In [5]:
X_test = test_data.drop(columns= ['PassengerId','Name'])
y_pred = model.predict(X_test)
submission = pd.DataFrame({
    'PassengerId':test_data['PassengerId'],
    'Survived': y_pred
})

submission.to_csv('cat_boot.csv', index=False)
print("Submission saved! You can now submit 'submission.csv' to Kaggle.")


Submission saved! You can now submit 'submission.csv' to Kaggle.
